In [1]:
# Import TensorFlow and TensorFlow Datasets
import plaidml.keras
plaidml.keras.install_backend()
import keras

import tensorflow_datasets as tfds
import tensorflow as tf

import os

In [2]:
print(keras.backend.backend())
print(tf.__version__)

plaidml
2.4.1


In [4]:
import numpy as np

In [5]:
import random

In [6]:
random.uniform(0,1)

0.3278367751011333

In [12]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = datasets['train'], datasets['test']

In [13]:
# You can also do info.splits.total_num_examples to get the total
# number of examples in the dataset.

num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [14]:
x = np.arange(2 * 2 * 3).reshape((2,2,3))
print(x, end="\n#####################\n")
print(np.flipud(x), end="\n#####################\n")
print(np.fliplr(x), end="\n#####################\n")
print(tf.image.flip_up_down(x), end="\n#####################\n")
print(tf.image.flip_left_right(x), end="\n#####################\n")

[[[ 0  1  2]
  [ 3  4  5]]

 [[ 6  7  8]
  [ 9 10 11]]]
#####################
[[[ 6  7  8]
  [ 9 10 11]]

 [[ 0  1  2]
  [ 3  4  5]]]
#####################
[[[ 3  4  5]
  [ 0  1  2]]

 [[ 9 10 11]
  [ 6  7  8]]]
#####################
tf.Tensor(
[[[ 6  7  8]
  [ 9 10 11]]

 [[ 0  1  2]
  [ 3  4  5]]], shape=(2, 2, 3), dtype=int64)
#####################
tf.Tensor(
[[[ 3  4  5]
  [ 0  1  2]]

 [[ 9 10 11]
  [ 6  7  8]]], shape=(2, 2, 3), dtype=int64)
#####################


In [68]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, ds,
                 horizontal_flip=False,
                 vertical_flip=False,
                 rescale=None,
                 shuffle=False,
                 batch_size=32,
                 num_classes=None,
                 steps_per_epoch=None):
        if steps_per_epoch == None:
            self.length = ds.__len__().numpy() // batch_size
        else:
            self.length = steps_per_epoch
        self.batch_size = batch_size
        if rescale == None:
            rescale = 1.
        self.image = list(map(lambda d: d[0]/rescale, ds.as_numpy_iterator()))
#         self.label = list(map(lambda d: keras.utils.np_utils.to_categorical(d[1], num_classes),
#                               ds.as_numpy_iterator()))
        self.label = list(map(lambda d: d[1],
                              ds.as_numpy_iterator()))
        self.num_examples = len(self.image)
        self.shuffle = shuffle
        self.horizontal_flip = horizontal_flip
        self.vertical_flip = vertical_flip
        self.num_classes = num_classes
        
    def __len__(self):
        return self.length
        
    def __getitem__(self, index):
        batch_x = self.image[index * self.batch_size : min(self.num_examples - 1, (index+1) * self.batch_size)]
        batch_y = self.label[index * self.batch_size : min(self.num_examples - 1, (index+1) * self.batch_size)]
        
        batch_x = np.array([self._preprocess(image) for image in batch_x])
        batch_y = np.array(batch_y)
        
        return batch_x, batch_y

    def on_epoch_end(self):
        if shuffle:
            new_image, new_label = zip(*random.sample(list(zip(self.image, self.label)), self.num_examples))
            self.image = list(new_image)
            self.label = list(new_label)
        
    def _preprocess(self, image):
        if self.horizontal_flip and random.uniform(0, 1) < 0.5:
            image = np.fliplr(image)
        if self.vertical_flip and random.uniform(0, 1) < 0.5:
            image = np.flipud(image)
        return image

In [69]:
train_generator = DataGenerator(mnist_train, rescale=255., num_classes=10)

In [70]:
for d in next(iter(train_generator)):
    print(d[0].shape)

(28, 28, 1)
()


In [71]:
model = keras.Sequential([
    keras.layers.Conv2D(32, 3, input_shape=(28, 28, 1)),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(64),
    keras.layers.Activation('relu'),
    keras.layers.Dense(10),
    keras.layers.Activation('softmax')
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [72]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_17 (Activation)   (None, 26, 26, 32)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 64)                346176    
_________________________________________________________________
activation_18 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 10)                650       
__________

In [73]:
model.input

<tile.Value conv2d_8_input Placeholder FLOAT32(<tile.Value SymbolicDim UINT64()>, 28, 28, 1)>

In [74]:
model.output

<tile.Value Softmax FLOAT32(<tile.Value SymbolicDim UINT64()>, 10)>

In [75]:
model.fit_generator(train_generator,
                    steps_per_epoch=None,
                    epochs=12,
                    verbose=1,
                    callbacks=None,
                    validation_data=None,
                    validation_steps=None,
                    class_weight=None,
                    max_queue_size=10,
                    workers=1,
                    use_multiprocessing=False,
                    shuffle=True,
                    initial_epoch=0)

Epoch 1/12


ERROR:plaidml:Compiler::Build> Compilation failure:
Compilation failed: 

program_source:47:7: warning: unused variable 'tid'
  int tid = _tid;
      ^
program_source:97:7: warning: unused variable 'tid'
  int tid = _tid;
      ^
program_source:182:7: warning: unused variable 'tid'
  int tid = _tid;
      ^
program_source:601:9: warning: unused variable 'x0_tid'
    int x0_tid = (tid % 32);
        ^
program_source:886:14: error: assigning to 'bool2' (vector of 2 'bool' values) from incompatible type 'metal::float2' (aka 'float2')
    agg[0] = select((float2)agg[0], (float2)agg_rhs, (bool2)c_cond != (bool2)0);
             ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
program_source:3384:7: warning: unused variable 'tid'
  int tid = _tid;
      ^
program_source:3756:9: warning: unused variable 'x1'
    int x1 = select((int)0, (int)x1_tid, (int)x1_cond);
        ^
program_source:3765:11: warning: unused variable 'LX_T173'
    float LX_T173 = agg[0];
          ^

ERR

Unknown: Compilation failed: 

program_source:47:7: warning: unused variable 'tid'
  int tid = _tid;
      ^
program_source:97:7: warning: unused variable 'tid'
  int tid = _tid;
      ^
program_source:182:7: warning: unused variable 'tid'
  int tid = _tid;
      ^
program_source:601:9: warning: unused variable 'x0_tid'
    int x0_tid = (tid % 32);
        ^
program_source:886:14: error: assigning to 'bool2' (vector of 2 'bool' values) from incompatible type 'metal::float2' (aka 'float2')
    agg[0] = select((float2)agg[0], (float2)agg_rhs, (bool2)c_cond != (bool2)0);
             ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
program_source:3384:7: warning: unused variable 'tid'
  int tid = _tid;
      ^
program_source:3756:9: warning: unused variable 'x1'
    int x1 = select((int)0, (int)x1_tid, (int)x1_cond);
        ^
program_source:3765:11: warning: unused variable 'LX_T173'
    float LX_T173 = agg[0];
          ^


In [85]:
train_image = np.array(list(map(lambda d: d[0]/255., mnist_train.as_numpy_iterator())))
# train_label = np.array(list(map(lambda d: keras.utils.np_utils.to_categorical(d[1], 10), mnist_train.as_numpy_iterator())))
train_label = np.array(list(map(lambda d: d[1], mnist_train.as_numpy_iterator())))

In [86]:
print(train_image.shape)
print(train_label.shape)

(60000, 28, 28, 1)
(60000,)


In [87]:
model.fit(train_image, train_label)

Epoch 1/1


ERROR:plaidml:Compiler::Build> Compilation failure:
Compilation failed: 

program_source:47:7: warning: unused variable 'tid'
  int tid = _tid;
      ^
program_source:97:7: warning: unused variable 'tid'
  int tid = _tid;
      ^
program_source:182:7: warning: unused variable 'tid'
  int tid = _tid;
      ^
program_source:601:9: warning: unused variable 'x0_tid'
    int x0_tid = (tid % 32);
        ^
program_source:886:14: error: assigning to 'bool2' (vector of 2 'bool' values) from incompatible type 'metal::float2' (aka 'float2')
    agg[0] = select((float2)agg[0], (float2)agg_rhs, (bool2)c_cond != (bool2)0);
             ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
program_source:3384:7: warning: unused variable 'tid'
  int tid = _tid;
      ^
program_source:3756:9: warning: unused variable 'x1'
    int x1 = select((int)0, (int)x1_tid, (int)x1_cond);
        ^
program_source:3765:11: warning: unused variable 'LX_T173'
    float LX_T173 = agg[0];
          ^

ERR

Unknown: Compilation failed: 

program_source:47:7: warning: unused variable 'tid'
  int tid = _tid;
      ^
program_source:97:7: warning: unused variable 'tid'
  int tid = _tid;
      ^
program_source:182:7: warning: unused variable 'tid'
  int tid = _tid;
      ^
program_source:601:9: warning: unused variable 'x0_tid'
    int x0_tid = (tid % 32);
        ^
program_source:886:14: error: assigning to 'bool2' (vector of 2 'bool' values) from incompatible type 'metal::float2' (aka 'float2')
    agg[0] = select((float2)agg[0], (float2)agg_rhs, (bool2)c_cond != (bool2)0);
             ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
program_source:3384:7: warning: unused variable 'tid'
  int tid = _tid;
      ^
program_source:3756:9: warning: unused variable 'x1'
    int x1 = select((int)0, (int)x1_tid, (int)x1_cond);
        ^
program_source:3765:11: warning: unused variable 'LX_T173'
    float LX_T173 = agg[0];
          ^


In [88]:
print(model.predict(train_image[-2:-1]))

[[0.10122783 0.089098   0.08580226 0.09582903 0.09731039 0.10844275
  0.10778651 0.09073113 0.12372182 0.10005023]]
